In [8]:
import os
from pathlib import Path
from dotenv import load_dotenv

ENV_FILE = f'../.env.'
load_dotenv(dotenv_path=Path(ENV_FILE), override=True)

username=os.getenv('username')
password=os.getenv('password')

print(f'{username}, {password}')

xiayy860612@126.com111, xiayy@1099


In [ ]:
import requests
from requests.auth import HTTPBasicAuth

session = requests.Session()
session.auth = HTTPBasicAuth(username, password)
response = session.post("https://api.worldquantbrain.com/authentication")

print(response.status_code)
print(response.json())

201
{'user': {'id': 'AX49862'}, 'token': {'expiry': 14400.0}, 'permissions': ['TUTORIAL']}


In [ ]:
simulation_data = {
    'type':'REGULAR',
    'settings':{
        'instrumentType':'EQUITY',
        'region':'USA',
        'universe':'TOP3000',
        'delay':1,
        'decay':0,
        'neutralization':'INDUSTRY',
        'truncation':0.08,
        'pasteurization':'ON',
        'unitHandling':'VERIFY',
        'nanHandling':'OFF',
        'language':'FASTEXPR',
        'visualization':False
    },
    'regular': 'liabilities/assets'
}

from time import sleep

sim_resp = session.post(
    'https://api.worldquantbrain.com/simulations',
    json = simulation_data,
)
sim_progress_url = sim_resp.headers['Location']

while True:
    sim_progress_resp = session.get(sim_progress_url)
    retry_after_sec = float(sim_progress_resp.headers.get("retry-After",0))
    if retry_after_sec == 0 :
        break
    sleep(retry_after_sec)

response = sim_progress_resp.json()
print(response)

alpha_id = response["alpha"]
print(f"Alpha ID: {alpha_id}")

{'id': '3crPxZ9Eb4gWbUF8NP4vTz7', 'type': 'REGULAR', 'settings': {'instrumentType': 'EQUITY', 'region': 'USA', 'universe': 'TOP3000', 'delay': 1, 'decay': 0, 'neutralization': 'INDUSTRY', 'truncation': 0.08, 'pasteurization': 'ON', 'unitHandling': 'VERIFY', 'nanHandling': 'OFF', 'maxTrade': 'OFF', 'language': 'FASTEXPR', 'visualization': False}, 'regular': 'liabilities/assets', 'status': 'COMPLETE', 'alpha': 'XAaQR0x'}
Alpha ID: XAaQR0x


In [3]:
def submit_alpha(alpha_id):
    url = f"https://api.worldquantbrain.com/alphas/{alpha_id}/submit"
    submit_response = session.post(url)
    while True:
        submit_response = session.get(sim_progress_url)
        retry_after_sec = float(submit_response.headers.get("retry-After",0))
        if retry_after_sec == 0:
            print("Alpha submitted successfully.")
            print(submit_response.status_code)
            print(submit_response.json())
            break
        sleep(retry_after_sec)
        # if 'retry-after' in submit_response.headers:
        #     retry_after_sec = float(submit_response.headers['retry-after'])
        #     sleep(retry_after_sec)
        #     submit_response = session.get(url)
        #     print(submit_response.status_code)
        #     print(submit_response.json())
        # else:
        #     break
submit_alpha('XAaQR0x')

Alpha submitted successfully.
200
{'id': '3crPxZ9Eb4gWbUF8NP4vTz7', 'type': 'REGULAR', 'settings': {'instrumentType': 'EQUITY', 'region': 'USA', 'universe': 'TOP3000', 'delay': 1, 'decay': 0, 'neutralization': 'INDUSTRY', 'truncation': 0.08, 'pasteurization': 'ON', 'unitHandling': 'VERIFY', 'nanHandling': 'OFF', 'maxTrade': 'OFF', 'language': 'FASTEXPR', 'visualization': False}, 'regular': 'liabilities/assets', 'status': 'COMPLETE', 'alpha': 'XAaQR0x'}
